<a href="https://colab.research.google.com/github/JoaoADuarte/Cardapio/blob/main/C%C3%B3pia_de_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium transformers yake google-auth
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium
!pip install transformers datasets peft
!pip install evaluate
!pip install rouge_score
!pip install numpy
!pip install datasets
!pip install torch
!pip install NLTK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 29.0 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:3 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:4 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
if 'google.colab' in str(get_ipython()):
    try:
        from google.colab import userdata
        hf_token = userdata.get('HF_TOKEN')
    except ImportError:
        print("Tentando usar google.colab.userdata, mas não encontrado.")
        hf_token = None # Ou defina um valor padrão ou lance um erro


In [ ]:
import os
import time
import sqlite3
import torch
import yake
import random # Adicionado para variar perguntas
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from transformers import T5ForConditionalGeneration, T5Tokenizer
from huggingface_hub import login, HfFolder
import re
from datasets import Dataset

# --- Configuração Inicial ---
hf_token = os.getenv("HF_TOKEN")


if hf_token:
    print("🔑 Token Hugging Face encontrado. Fazendo login...")
    login(token=hf_token)
    # Salva o token para uso futuro na sessão/máquina
    HfFolder.save_token(hf_token)
else:
    print("⚠️ Token Hugging Face (HF_TOKEN) não encontrado nas variáveis de ambiente ou segredos do Colab.")
    print("   O script pode falhar ao interagir com modelos privados ou ao fazer uploads.")
    # login() # Isso pedirá o token interativamente no terminal

# Caminho do banco de dados
db_path = "/content/drive/MyDrive/chatbot_jardinagem/chatbot_jardinagem_novo2.db" if os.path.exists("/content/drive") else "chatbot_jardinagem_novo2.db"
print(f"💾 Usando banco de dados em: {db_path}")

# YAKE Keyword Extractor
kw_extrator = yake.KeywordExtractor(lan="pt", n=1, top=10) # Extrai 10 palavras-chave de 1 termo

# Selenium WebDriver Options
chrome_options = Options()
chrome_options.add_argument("--headless") # Roda sem abrir janela do navegador
chrome_options.add_argument("--no-sandbox") # Necessário em alguns ambientes Linux/Docker
chrome_options.add_argument("--disable-dev-shm-usage") # Evita problemas de memória compartilhada
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36") # Define um user-agent comum
driver = webdriver.Chrome(options=chrome_options)
print("🚀 WebDriver do Selenium inicializado.")

# Modelo T5 para Resumo (usado no scraping)
tokenizer_resumo = T5Tokenizer.from_pretrained("t5-base")
model_resumo = T5ForConditionalGeneration.from_pretrained("t5-base")
print("🧠 Modelo T5-base para resumo carregado.")

# --- Banco de Dados ---
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS artigos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fonte TEXT,
    titulo TEXT UNIQUE, -- Adicionado UNIQUE para garantir títulos únicos
    conteudo_resumido TEXT, -- Renomeado para clareza
    conteudo_completo TEXT, -- Nova coluna para o texto integral
    tags TEXT,
    data_extracao TEXT
);""")
#  Adiciona índice no título para buscas mais rápidas
cursor.execute("CREATE INDEX IF NOT EXISTS idx_titulo ON artigos (titulo);")
conn.commit()
print("📦 Tabela 'artigos' no banco de dados verificada/criada.")

#  Função de inserção
def inserir_dados(fonte, titulo, conteudo_resumido, conteudo_completo, tags):
    """Insere os dados coletados no banco de dados, evitando duplicatas de título."""
    if not conteudo_completo or len(conteudo_completo.strip()) < 150: # Aumentei um pouco o limite mínimo
        print(f"⚠️ Conteúdo completo muito curto ({len(conteudo_completo.strip())} chars). Ignorado: {titulo[:60]}...")
        return False
    if not titulo:
        print("⚠️ Título vazio. Ignorado.")
        return False

    try:
        cursor.execute("""INSERT INTO artigos (fonte, titulo, conteudo_resumido, conteudo_completo, tags, data_extracao)
                          VALUES (?, ?, ?, ?, ?, ?)""",
                       (fonte, titulo.strip(), conteudo_resumido.strip(), conteudo_completo.strip(), tags, datetime.now().isoformat()))
        conn.commit()
        print(f"✅ Inserido: {titulo[:60]}...")
        return True
    except sqlite3.IntegrityError: # Erro específico para violação de UNIQUE (título duplicado)
        print(f"⚠️ Título já existente. Ignorado: {titulo[:60]}...")
        return False
    except sqlite3.Error as e:
        print(f"❌ Erro ao inserir '{titulo[:60]}...': {e}")
        return False

# --- Funções Auxiliares ---

def resumir_texto(texto, max_len=512, summary_max_len=150, summary_min_len=50):
    """Gera um resumo do texto usando o modelo T5."""
    if not texto:
        return ""
    try:
        input_text = f"summarize: {texto}"
        inputs = tokenizer_resumo(input_text, return_tensors="pt", max_length=max_len, truncation=True)
        with torch.no_grad():
            summary_ids = model_resumo.generate(
                inputs["input_ids"],
                max_length=summary_max_len,
                min_length=summary_min_len,
                num_beams=4, # Beam search para melhor qualidade
                early_stopping=True # Para de gerar se o modelo achar que terminou
            )
        return tokenizer_resumo.decode(summary_ids[0], skip_special_tokens=True)
    except Exception as e:
        print(f"⚠️ Erro ao resumir texto: {e}")
        return "" # Retorna vazio em caso de erro

def encontrar_titulo_com_wait(driver, timeout=10):
    """Tenta encontrar o título da página usando uma lista de seletores CSS comuns."""
    # Lista de seletores comuns para títulos
    seletores_titulo = [
        "h1",
        ".entry-title", ".post-title", "#page-title", ".tdb-title-text",
        ".post-header h1.title", "header h1", ".headline", ".article-title"
        ]
    for seletor in seletores_titulo:
        try:
            # Espera o elemento estar presente
            titulo_elemento = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, seletor))
            )
            # Verifica se o texto não está vazio
            texto_titulo = titulo_elemento.text.strip()
            if texto_titulo:
                return texto_titulo
        except TimeoutException:
            continue # Tenta o próximo seletor
        except Exception as e: # Outros erros possíveis (elemento obsoleto, etc.)
             print(f"🔍 Erro ao tentar seletor de título '{seletor}': {e}")
             continue
    print("⚠️ Não foi possível encontrar um título com os seletores padrão.")
    # Como último recurso, tenta pegar o <title> da página
    try:
        return driver.title.strip()
    except Exception:
        return None # Retorna None se falhar

#  Extração de conteúdo mais específica**
def extrair_conteudo_principal(driver, timeout=10):
    """Tenta extrair parágrafos do contêiner principal do artigo."""
    # Lista de seletores comuns para o corpo/conteúdo principal do artigo
    seletores_conteudo = [
        "article .entry-content", "article .post-content", "article", # Estruturas comuns com <article>
        ".td-post-content", ".pf-content", ".content", ".post-body", ".materia-conteudo", # Classes comuns
        "#content", "#main", "#main-content", "#articleBody" # IDs comuns
    ]
    container_principal = None
    for seletor in seletores_conteudo:
        try:
            # Espera o container principal estar presente
            container_principal = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, seletor))
            )
            print(f"🔎 Encontrado container principal com seletor: '{seletor}'")
            break # Usa o primeiro que encontrar
        except TimeoutException:
            continue
        except Exception as e:
             print(f"🔍 Erro ao tentar seletor de conteúdo '{seletor}': {e}")
             continue

    paragrafos_texto = []
    if container_principal:
        # Encontra parágrafos DENTRO do container principal
        paragrafos = container_principal.find_elements(By.TAG_NAME, "p")
        if not paragrafos:
             # Se não achar <p>, tenta pegar todo texto do container (menos ideal)
             print("⚠️ Não foram encontrados <p> no container, pegando texto geral.")
             texto_completo = container_principal.text.strip()
             # Quebra em linhas como aproximação de parágrafos
             paragrafos_texto = [linha for linha in texto_completo.split('\n') if linha.strip()]
        else:
             paragrafos_texto = [p.text.strip() for p in paragrafos if p.text and len(p.text.strip()) > 20] # Filtra parágrafos curtos/vazios
    else:
        # Fallback: Se NENHUM container principal for encontrado, pega todos os <p> da página
        print("⚠️ Container principal não encontrado. Usando fallback: todos os <p> da página.")
        try:
            paragrafos = WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, "p"))
            )
            paragrafos_texto = [p.text.strip() for p in paragrafos if p.text and len(p.text.strip()) > 20]
        except TimeoutException:
            print("⚠️ Tempo esgotado ao buscar todos os <p> como fallback.")
            return "" # Retorna vazio se o fallback falhar

    # Junta os parágrafos filtrados
    conteudo_final = "\n".join(paragrafos_texto)

    # Limpeza básica adicional
    conteudo_final = re.sub(r'(?i)^\s*(leia mais|veja também|relacionado):?.*$', '', conteudo_final, flags=re.MULTILINE)
    conteudo_final = re.sub(r'\n{3,}', '\n\n', conteudo_final) # Remove excesso de linhas em branco

    return conteudo_final.strip()

# URLs de scraping
urls = [
    "https://armazemagricola.pt/como-plantar-tomates/",
    "https://www.terra.com.br/vida-e-estilo/casa-e-decoracao/como-plantar-tomate-em-casa,f0a08517c124e92dc78d5c52650375a8hioufx0h.html",
    "https://eos.com/pt/blog/como-plantar-tomates/",
    "https://ciclovivo.com.br/mao-na-massa/horta/como-plantar-tomate-organico-em-casa/",
    "https://global.tramontina.com/blog/faca-voce-mesmo/como-plantar-tomate-em-casa",
    "https://www.cpt.com.br/artigos/horta-como-plantar-tomates-solanum-lycopersicum",
    "https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html",
    "https://www.petz.com.br/blog/plantas/como-plantar-tomate/",
    "https://jardimdasideias.com.br/saiba-quais-as-melhores-hortalicas-para-cultivar-em-fevereiro/",
    "https://jardimdasideias.com.br/plantas-ideais-para-um-jardim-de-baixa-manutencao/",
    "https://jardimdasideias.com.br/guia-pratico-para-iniciantes-na-jardinagem-como-comecar-e-cuidar-do-seu-jardim/",
    "https://jardimdasideias.com.br/dicas-praticas-para-transformar-o-seu-jardim/",
    "https://www.jardineiro.net/40-plantas-floridas-para-canteiros-no-sol.html",
    "https://www.jardineiro.net/40-plantas-com-flor-vermelha-para-um-jardim-encantador.html",
    "https://www.jardineiro.net/8-causas-de-folhas-com-pontas-secas-e-queimadas.html",
    "https://www.jardineiro.net/suculentas-o-guia-completo-para-iniciantes.html",
    "https://www.jardineiro.net/classe/plantas-flutuantes",
    "https://www.jardineiro.net/introducao-ao-bonsai.html",
    "https://www.jardineiro.net/a-arte-japonesa-na-admiracao-da-natureza.html",
    "https://www.jardineiro.net/pragas/percevejo-do-sul-blissus-insularis.html",
    "https://www.jardineiro.net/pragas/acaros.html",
    "https://www.jardineiro.net/pragas/moscas-dos-fungos-fungus-gnats.html",
    "https://www.jardineiro.net/pragas/antracnose.html",
    "https://www.jardineiro.net/pragas/cupim-de-grama.html"
]
urls = list(set(urls)) # Remove duplicatas da lista

print(f"\n--- Iniciando Coleta de {len(urls)} URLs ---")
artigos_inseridos = 0
for url in urls:
    print(f"\n🌐 Coletando de: {url}")
    try:
        driver.get(url)
        # Pequena espera para elementos dinâmicos iniciais carregarem (opcional)
        time.sleep(1) # Pode ajustar ou remover se os waits explícitos forem suficientes

        titulo = encontrar_titulo_com_wait(driver)
        if not titulo:
            print(f"⚠️ Não foi possível encontrar um título adequado em {url}. Pulando.")
            continue

        # **MELHORIA: Usa a função de extração de conteúdo principal**
        conteudo_completo = extrair_conteudo_principal(driver)

        if not conteudo_completo or len(conteudo_completo) < 200: # Checa o conteúdo completo agora
            print("⚠️ Conteúdo completo insuficiente após extração.")
            continue

        # Gera resumo a partir do conteúdo completo
        resumo = resumir_texto(conteudo_completo)
        if not resumo:
             print("⚠️ Falha ao gerar resumo, mas continuando com conteúdo completo.")
             resumo = "" # Garante que não é None

        # Extrai tags do conteúdo completo
        tags = ", ".join([kw for kw, _ in kw_extrator.extract_keywords(conteudo_completo)])

        # **MELHORIA: Passa o conteúdo completo para inserção**
        if inserir_dados(url, titulo, resumo, conteudo_completo, tags):
            artigos_inseridos += 1

        # Pausa leve para não sobrecarregar os servidores
        time.sleep(random.uniform(1.5, 3.0))

    except TimeoutException as e:
        print(f"⚠️ Tempo de espera excedido ao carregar/interagir com a página {url}: {e}")
    except Exception as e:
        print(f"❌ Erro inesperado ao processar {url}: {e}")
        # Tenta reiniciar o driver em caso de erros graves
        # driver.quit()
        # driver = webdriver.Chrome(options=chrome_options)

# --- Finalização do Scraping ---
driver.quit()
print("\n--- Coleta Finalizada ---")
print(f"📰 Artigos inseridos com sucesso nesta execução: {artigos_inseridos}")

# --- Preparação do Dataset para Treinamento ---

print("\n--- Iniciando Preparação do Dataset para Treinamento ---")

def limpar_conteudo_treino(conteudo):
    """Limpa o texto removendo ruídos comuns para usar no dataset."""
    if not conteudo: return ""
    # Remove múltiplos espaços/newlines
    conteudo = re.sub(r"[\s\n\r\t]+", " ", conteudo)
    # Remove URLs
    conteudo = re.sub(r"https?://\S+|www\.[a-zA-Z0-9\-\.]+\.[a-zA-Z]+", "", conteudo)
     # Remove frases de boilerplate comuns (case-insensitive)
    boilerplate = [
        r"usamos cookies[^.]*\.", r"compartilhe[^.]*\.", r"política de privacidade[^.]*\.",
        r"leia mais[^.]*\.", r"veja também[^.]*\.", r"fontes?:[^.]*\.", r"imagens?:[^.]*\.",
        r"créditos?:[^.]*\.", r"^\s*publicidade\s*$", r"assine nossa newsletter"
        ]
    for pattern in boilerplate:
        conteudo = re.sub(pattern, "", conteudo, flags=re.IGNORECASE)
    # Remove caracteres não essenciais (mantém pontuação básica e acentos pt-BR)
    # conteudo = re.sub(r"[^a-zA-Z0-9çáéíóúãõâêîôûàèìòùüÁÉÍÓÚÃÕÂÊÎÔÛÀÈÌÒÙÜ\s,.!?-]", "", conteudo)
    # Versão um pouco mais permissiva (mantém %)
    conteudo = re.sub(r"[^\w\s,.!?-çáéíóúãõâêîôûàèìòùüÁÉÍÓÚÃÕÂÊÎÔÛÀÈÌÒÙÜ%]", "", conteudo, flags=re.UNICODE)
    # Remove espaços extras novamente após remoções
    conteudo = re.sub(r"\s{2,}", " ", conteudo).strip()
    return conteudo

# *Geração de perguntas mais variadas
def gerar_pergunta_treino(titulo, tags_str):
    """Gera uma pergunta mais variada baseada no título e opcionalmente nas tags."""
    titulo_limpo = titulo.strip().lower()
    tags = [tag.strip().lower() for tag in tags_str.split(',')] if tags_str else []

    # Padrões de pergunta
    perguntas_como_plantar = [
        "Como plantar {item}?",
        "Quais são os passos para plantar {item}?",
        "Explique como cultivar {item}.",
        "Qual a melhor forma de plantar {item} em casa?",
        "Dicas para plantar {item}?",
    ]
    perguntas_genericas = [
        "Fale sobre {item}.",
        "O que você sabe sobre {item}?",
        "Explique o artigo sobre {item}.",
        "Resuma as informações sobre {item}.",
    ]
    perguntas_pragas_doencas = [
        "Como combater {item}?",
        "Qual o tratamento para {item} em plantas?",
        "Como identificar e lidar com {item} no jardim?",
    ]

    # Tenta extrair o item principal do título
    item = titulo_limpo
    if "como plantar" in titulo_limpo:
        item = titulo_limpo.split("como plantar")[-1].strip().replace("?", "")
        return random.choice(perguntas_como_plantar).format(item=item.capitalize())
    elif "plantar" in titulo_limpo:
         # Tenta ser um pouco mais esperto para pegar o item
         match = re.search(r"(plantar|cultivar)\s+(.+)", titulo_limpo)
         if match:
             item = match.group(2).strip().replace("?", "")
             return random.choice(perguntas_como_plantar).format(item=item.capitalize())
         else: # Fallback se a regex falhar
             item = titulo_limpo # Usa o título todo
             return random.choice(perguntas_genericas).format(item=item.capitalize())
    elif any(p in tags for p in ["praga", "pragas", "doença", "doenças", "inseto", "fungo", "ácaro", "cupim", "percevejo"]):
        # Se o título não for sobre plantar, mas as tags indicam praga/doença
         # Tenta pegar o nome da praga/doença do título se possível
         match = re.search(r"(?:praga[s]?|doença[s]?|combater|tratar)\s*[:\-]?\s*(.+)", titulo_limpo)
         if match:
             item = match.group(1).strip()
             return random.choice(perguntas_pragas_doencas).format(item=item.capitalize())
         else: # Se não conseguir isolar, usa o título
             item = titulo_limpo
             return random.choice(perguntas_genericas).format(item=item.capitalize()) # Ou uma pergunta mais específica sobre o artigo
    else:
        # Fallback genérico para outros títulos
        return random.choice(perguntas_genericas).format(item=titulo_limpo.capitalize())


def carregar_dados_do_banco_treino(db_path):
    """Carrega título, conteúdo completo e tags do banco para criar o dataset."""
    dataset_lista = []
    try:
        conn_train = sqlite3.connect(db_path)
        cursor_train = conn_train.cursor()
        # **MELHORIA: Seleciona conteudo_completo e tags**
        cursor_train.execute("SELECT titulo, conteudo_completo, tags FROM artigos WHERE conteudo_completo IS NOT NULL")
        dados_db = cursor_train.fetchall()
        conn_train.close()
        print(f"📄 Carregados {len(dados_db)} registros do banco para gerar dataset.")

        for titulo, conteudo_completo, tags in dados_db:
             # **MELHORIA: Usa a função de geração aprimorada**
            pergunta = gerar_pergunta_treino(titulo, tags)
            # **MELHORIA: Limpa e trunca o conteúdo completo**
            resposta = limpar_conteudo_treino(conteudo_completo)

            # Trunca a resposta para um tamanho máximo gerenciável pelo modelo
            max_len_resposta = 500 # Ajuste conforme necessário
            resposta = resposta[:max_len_resposta]

            # Garante que pergunta e resposta não sejam muito curtas
            if pergunta and len(resposta) >= 100: # Mínimo de 100 caracteres para a resposta
                dataset_lista.append({"pergunta": pergunta, "resposta": resposta})
            # else:
            #     print(f"⚠️ Registro ignorado por resposta curta ou pergunta vazia: Título '{titulo[:50]}...'")


        print(f"📊 Gerados {len(dataset_lista)} pares pergunta/resposta válidos.")
        return dataset_lista
    except sqlite3.Error as e:
        print(f"❌ Erro ao acessar o banco de dados para treino: {e}")
        return []

# Carregar dados diretamente do banco de dados
dados_treino = carregar_dados_do_banco_treino(db_path)

if not dados_treino:
    print("🛑 Nenhum dado de treinamento gerado. Verifique o banco de dados ou os filtros.")
else:
    # Converter para Dataset do Hugging Face
    dataset_treino_hf = Dataset.from_list(dados_treino)

    # Dividir o dataset em treino e teste (ex: 80% treino, 20% teste)
    if len(dataset_treino_hf) > 10: # Só divide se tiver dados suficientes
      dataset_treino_split = dataset_treino_hf.train_test_split(test_size=0.2, seed=42) # seed para reprodutibilidade
      train_dataset = dataset_treino_split["train"]
      test_dataset = dataset_treino_split["test"]
    else:
       print("⚠️ Dataset muito pequeno, usando tudo para treino.")
       train_dataset = dataset_treino_hf
       test_dataset = None # Ou criar um teste vazio/pequeno se a lib de treino exigir

    print(f" splitting em {len(train_dataset)} amostras de treino" + (f" e {len(test_dataset)} de teste." if test_dataset else "."))

    # Carregar modelo e tokenizador para o Flan-T5 (ou outro modelo de sua escolha)
    modelo_nome_ia = "google/flan-t5-large" # Pode experimentar 'base' ou 'small' para mais rápido
    print(f"🔄 Carregando tokenizer e modelo: {modelo_nome_ia}...")
    try:
        tokenizer_ia = T5Tokenizer.from_pretrained(modelo_nome_ia)
        # model_ia = T5ForConditionalGeneration.from_pretrained(modelo_nome_ia) # Descomente se precisar do modelo aqui
        print("✅ Tokenizer carregado.")
    except Exception as e:
        print(f"❌ Falha ao carregar tokenizer/modelo {modelo_nome_ia}: {e}")
        print("   Verifique o nome do modelo e sua conexão com a Hugging Face.")
        # Encerrar ou tratar o erro aqui
        exit()

    # Função para pré-processar as amostras para o Flan-T5
    def preprocessar_amostra_ia(exemplos):
        """Formata e tokeniza os dados para o formato esperado pelo T5."""
        # Formato comum para fine-tuning T5 em tarefas de QA ou instrução
        prefixo = "Responda a pergunta com base no contexto de jardinagem: " # Ou outro prefixo de instrução
        entradas = [prefixo + "Pergunta: " + str(p) for p in exemplos["pergunta"]]
        # Tokeniza as entradas (perguntas com prefixo)
        entradas_tokenizadas = tokenizer_ia(
            entradas,
            padding="max_length", # Preenche até max_length
            truncation=True,      # Trunca se for maior
            max_length=512        # Máximo para o input do T5 (ajuste se necessário)
        )

        # Tokeniza as saídas (respostas) - estas serão os 'labels'
        respostas_tokenizadas = tokenizer_ia(
            text_target=exemplos["resposta"], # Argumento correto para labels no T5Tokenizer
            padding="max_length",
            truncation=True,
            max_length=128       # Máximo para a saída (resposta gerada) - ajuste conforme necessário
        )

        # Define os 'labels' para o cálculo da loss durante o treino
        entradas_tokenizadas["labels"] = respostas_tokenizadas["input_ids"]
        return entradas_tokenizadas

    # Aplica o pré-processamento aos datasets de treino e teste
    dataset_treino_processado = train_dataset.map(preprocessar_amostra_ia, batched=True)
    if test_dataset:
        dataset_teste_processado = test_dataset.map(preprocessar_amostra_ia, batched=True)

    # Salvar o dataset pré-processado
    save_path = "dataset_jardinagem_flant5_pronto"
    dataset_treino_processado.save_to_disk(os.path.join(save_path, "train"))
    if test_dataset:
        dataset_teste_processado.save_to_disk(os.path.join(save_path, "test"))
        print(f"✅ Datasets de treino e teste pré-processados e salvos em '{save_path}'")
    else:
         print(f"✅ Dataset de treino pré-processado e salvo em '{save_path}' (sem dataset de teste)")


# Fechar conexão com o banco de dados se ainda estiver aberta
if 'conn' in locals() and conn:
    conn.close()
    print("\n🚪 Conexão com o banco de dados fechada.")

print("\n🎉 Script finalizado.")

⚠️ Token Hugging Face (HF_TOKEN) não encontrado nas variáveis de ambiente ou segredos do Colab.
   O script pode falhar ao interagir com modelos privados ou ao fazer uploads.
💾 Usando banco de dados em: /content/drive/MyDrive/chatbot_jardinagem/chatbot_jardinagem_novo2.db
🚀 WebDriver do Selenium inicializado.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5 [00:00<?, ? examples/s]

✅ Datasets de treino e teste pré-processados e salvos em 'dataset_jardinagem_flant5_pronto'

🚪 Conexão com o banco de dados fechada.

🎉 Script finalizado.


Apenas para testar o token da hf

In [ ]:
import os
print(os.getenv("HF_TOKEN"))


None


In [ ]:
!python prepare_dataset.py --db_path /content/drive/MyDrive/caminho/diferente/meu_banco.db --output_path ./meu_dataset_processado --model_name google/flan-t5-large

python3: can't open file '/content/prepare_dataset.py': [Errno 2] No such file or directory


In [ ]:
!python prepare_dataset.py --output_path ./meu_dataset_jardinagem_processado

python3: can't open file '/content/prepare_dataset.py': [Errno 2] No such file or directory


Ajustes no pré-processamento e no F-T

In [ ]:
import os
import sqlite3
import re
import random
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer
import argparse # Para passar argumentos via linha de comando

# --- Funções Auxiliares (Limpeza e Geração de Perguntas) ---

def limpar_conteudo_treino(conteudo):
    """Limpa o texto removendo ruídos comuns para usar no dataset."""
    if not conteudo: return ""
    conteudo = re.sub(r"[\s\n\r\t]+", " ", conteudo)
    conteudo = re.sub(r"https?://\S+|www\.[a-zA-Z0-9\-\.]+\.[a-zA-Z]+", "", conteudo)
    boilerplate = [
        r"usamos cookies[^.]*\.", r"compartilhe[^.]*\.", r"política de privacidade[^.]*\.",
        r"leia mais[^.]*\.", r"veja também[^.]*\.", r"fontes?:[^.]*\.", r"imagens?:[^.]*\.",
        r"créditos?:[^.]*\.", r"^\s*publicidade\s*$", r"assine nossa newsletter"
    ]
    for pattern in boilerplate:
        conteudo = re.sub(pattern, "", conteudo, flags=re.IGNORECASE)
    conteudo = re.sub(r"[^\w\s,.!?-çáéíóúãõâêîôûàèìòùüÁÉÍÓÚÃÕÂÊÎÔÛÀÈÌÒÙÜ%]", "", conteudo, flags=re.UNICODE)
    conteudo = re.sub(r"\s{2,}", " ", conteudo).strip()
    return conteudo

def gerar_pergunta_treino(titulo, tags_str):
    """Gera uma pergunta mais variada baseada no título e opcionalmente nas tags."""
    titulo_limpo = titulo.strip().lower()
    tags = [tag.strip().lower() for tag in tags_str.split(',')] if tags_str else []
    perguntas_como_plantar = [
        "Como plantar {item}?", "Quais são os passos para plantar {item}?",
        "Explique como cultivar {item}.", "Qual a melhor forma de plantar {item} em casa?",
        "Dicas para plantar {item}?",
    ]
    perguntas_genericas = [
        "Fale sobre {item}.", "O que você sabe sobre {item}?",
        "Explique o artigo sobre {item}.", "Resuma as informações sobre {item}.",
        "O que o artigo '{item}' diz?"
    ]
    perguntas_pragas_doencas = [
        "Como combater {item}?", "Qual o tratamento para {item} em plantas?",
        "Como identificar e lidar com {item} no jardim?",
    ]
    item = titulo_limpo
    if "como plantar" in titulo_limpo:
        item = titulo_limpo.split("como plantar")[-1].strip().replace("?", "")
        return random.choice(perguntas_como_plantar).format(item=item.capitalize())
    elif "plantar" in titulo_limpo:
         match = re.search(r"(plantar|cultivar)\s+(.+)", titulo_limpo)
         if match:
             item = match.group(2).strip().replace("?", "")
             return random.choice(perguntas_como_plantar).format(item=item.capitalize())
         else:
             return random.choice(perguntas_genericas).format(item=titulo_limpo.capitalize())
    elif any(p in tags for p in ["praga", "pragas", "doença", "doenças", "inseto", "fungo", "ácaro", "cupim", "percevejo"]):
         match = re.search(r"(?:praga[s]?|doença[s]?|combater|tratar)\s*[:\-]?\s*(.+)", titulo_limpo)
         if match:
             item = match.group(1).strip()
             return random.choice(perguntas_pragas_doencas).format(item=item.capitalize())
         else:
             return random.choice(perguntas_genericas).format(item=titulo_limpo.capitalize())
    else:
        return random.choice(perguntas_genericas).format(item=titulo_limpo.capitalize())

# --- Função de Carregamento do Banco ---

def carregar_dados_brutos_banco(db_path, max_resposta_len=500, min_resposta_len=50):
    """Carrega dados do banco, gera pares pergunta/resposta e retorna lista de dicts."""
    dataset_lista = []
    conn = None
    try:
        if not os.path.exists(db_path):
             print(f"❌ Erro: Arquivo do banco de dados não encontrado em {db_path}")
             return []
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT titulo, conteudo_completo, tags FROM artigos WHERE conteudo_completo IS NOT NULL")
        dados_db = cursor.fetchall()
        print(f"📄 Carregados {len(dados_db)} registros do banco.")

        for titulo, conteudo_completo, tags in dados_db:
            pergunta = gerar_pergunta_treino(titulo, tags)
            resposta_limpa = limpar_conteudo_treino(conteudo_completo)
            resposta_truncada = resposta_limpa[:max_resposta_len]

            if pergunta and len(resposta_truncada) >= min_resposta_len:
                # Mantém pergunta/resposta originais para referência se precisar
                dataset_lista.append({
                    "id": f"{titulo[:50]}_{random.randint(1000,9999)}", # ID simples
                    "pergunta": pergunta,
                    "resposta": resposta_truncada
                    })
            # else:
            #     print(f"⚠️ Registro ignorado (resposta curta/pergunta vazia): Titulo '{titulo[:50]}...'")

        print(f"📊 Gerados {len(dataset_lista)} pares pergunta/resposta válidos.")
        return dataset_lista
    except sqlite3.Error as e:
        print(f"❌ Erro ao acessar o banco de dados: {e}")
        return []
    finally:
        if conn:
            conn.close()

# --- Função de Pré-processamento T5 ---

def preprocessar_dados_t5(exemplos, tokenizer, max_input_length=512, max_target_length=128):
    """Formata e tokeniza os dados para o T5, separando input e labels."""
    prefixo = "Responda a pergunta sobre jardinagem: "
    inputs = [prefixo + str(p) for p in exemplos["pergunta"]]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        padding="max_length",
        truncation=True
    )
    labels = tokenizer(
        text_target=exemplos["resposta"],
        max_length=max_target_length,
        padding="max_length",
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# --- Lógica Principal ---

def main(db_path, output_path, model_name, test_size=0.15, seed=42):
    """Função principal para preparar e salvar o dataset."""

    print("--- Iniciando Preparação do Dataset ---")

    # 1. Carregar dados brutos do banco
    dados_brutos = carregar_dados_brutos_banco(db_path)
    if not dados_brutos:
        print("🛑 Nenhum dado bruto encontrado ou gerado. Encerrando.")
        return

    # 2. Converter para Dataset Hugging Face
    dataset_hf = Dataset.from_list(dados_brutos)
    print(f"📊 Dataset Hugging Face criado com {len(dataset_hf)} exemplos.")

    # 3. Dividir em treino e teste
    if len(dataset_hf) > 10:
        dataset_splits = dataset_hf.train_test_split(test_size=test_size, seed=seed)
        train_dataset = dataset_splits["train"]
        eval_dataset = dataset_splits["test"]
        print(f" splitting em {len(train_dataset)} amostras de treino e {len(eval_dataset)} de teste.")
    else:
        print("⚠️ Dataset muito pequeno, usando tudo para treino e criando teste vazio.")
        train_dataset = dataset_hf
        # Cria um dataset de teste vazio com as mesmas features para evitar erros no Trainer depois
        # (alternativa é não ter avaliação no fine-tuning)
        eval_dataset = Dataset.from_dict(train_dataset.features.copy()).select([])


    # 4. Carregar Tokenizer (necessário para o pré-processamento)
    print(f"🔄 Carregando tokenizer para {model_name}...")
    try:
        tokenizer = T5Tokenizer.from_pretrained(model_name)
    except Exception as e:
        print(f"❌ Falha ao carregar tokenizer {model_name}: {e}")
        return

    # 5. Aplicar pré-processamento (Tokenização e formatação T5)
    print("⚙️ Aplicando pré-processamento aos dados (pode levar um tempo)...")
    # Colunas a remover após tokenização (as originais de texto)
    remove_cols = ["pergunta", "resposta", "id"] # Remove o ID também se não for usar

    train_dataset_processed = train_dataset.map(
        preprocessar_dados_t5,
        batched=True,
        fn_kwargs={"tokenizer": tokenizer},
        remove_columns=[col for col in remove_cols if col in train_dataset.column_names]
    )
    eval_dataset_processed = eval_dataset.map(
        preprocessar_dados_t5,
        batched=True,
        fn_kwargs={"tokenizer": tokenizer},
        remove_columns=[col for col in remove_cols if col in eval_dataset.column_names]
    )

    # 6. Agrupar em DatasetDict e Salvar
    processed_dataset_dict = DatasetDict({
        "train": train_dataset_processed,
        "test": eval_dataset_processed
    })

    print(f"💾 Salvando dataset processado em '{output_path}'...")
    try:
        processed_dataset_dict.save_to_disk(output_path)
        print(f"✅ Dataset processado e salvo com sucesso em '{output_path}'.")
        print("   Este dataset está pronto para ser usado no script de fine-tuning.")
    except Exception as e:
        print(f"❌ Erro ao salvar o dataset processado: {e}")




if __name__ == "__main__":
    # --- Configuração dos Argumentos ---
    parser = argparse.ArgumentParser(description="Prepara dataset para fine-tuning de modelo T5 a partir de um banco de dados SQLite.")
    parser.add_argument(
        "--db_path",
        type=str,
        # **MELHORIA: Define o nome do banco que você usa como padrão**
        default="chatbot_jardinagem_novo2.db",
        help="Caminho para o arquivo do banco de dados SQLite."
    )
    parser.add_argument(
        "--output_path",
        type=str,
        default="./processed_dataset_jardinagem", # Pasta padrão de saída
        help="Caminho da pasta onde o dataset processado será salvo."
    )
    parser.add_argument(
        "--model_name",
        type=str,
        default="google/flan-t5-large",
        help="Nome do modelo large T5 (ex: google/flan-t5-large) cujo tokenizer será usado."
    )
    parser.add_argument(
        "--test_size",
        type=float,
        default=0.15, # 15% para teste
        help="Proporção do dataset a ser usada para teste (validação)."
    )
    parser.add_argument(
        "--seed",
        type=int,
        default=42, # Seed para reprodutibilidade da divisão treino/teste
        help="Seed para a divisão treino/teste."
    )

    # **MELHORIA: Usa parse_known_args() para ignorar args extras do Colab/Jupyter**
    args, unknown_args = parser.parse_known_args()
    if unknown_args:
        print(f"⚠️ Argumentos desconhecidos ignorados: {unknown_args}")

    # Verifica se o db_path precisa ser ajustado para Google Drive
    db_full_path = args.db_path
    # Checa se o path fornecido NÃO existe E se o Google Drive está montado
    if not os.path.exists(db_full_path) and os.path.exists("/content/drive/MyDrive"):
         # Tenta montar o caminho completo no Drive (ajuste se sua estrutura for diferente)
         drive_path = f"/content/drive/MyDrive/chatbot_jardinagem/{os.path.basename(args.db_path)}"
         if os.path.exists(drive_path):
              print(f"Ajustando caminho do DB para Google Drive: {drive_path}")
              db_full_path = drive_path
         else:
              print(f"⚠️ Aviso: DB não encontrado em '{args.db_path}' nem em '{drive_path}'. Verifique o caminho.")
              # Decide se quer continuar ou parar
              # exit() # Descomente para parar se o DB não for encontrado

    main(db_full_path, args.output_path, args.model_name, args.test_size, args.seed)

⚠️ Argumentos desconhecidos ignorados: ['-f', '/root/.local/share/jupyter/runtime/kernel-b69d2cd5-011c-4b29-9374-0a7526f155ad.json']
Ajustando caminho do DB para Google Drive: /content/drive/MyDrive/chatbot_jardinagem/chatbot_jardinagem_novo2.db
--- Iniciando Preparação do Dataset ---
📄 Carregados 22 registros do banco.
📊 Gerados 22 pares pergunta/resposta válidos.
📊 Dataset Hugging Face criado com 22 exemplos.
 splitting em 18 amostras de treino e 4 de teste.
🔄 Carregando tokenizer para google/flan-t5-large...
⚙️ Aplicando pré-processamento aos dados (pode levar um tempo)...


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

💾 Salvando dataset processado em './processed_dataset_jardinagem'...


Saving the dataset (0/1 shards):   0%|          | 0/18 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

✅ Dataset processado e salvo com sucesso em './processed_dataset_jardinagem'.
   Este dataset está pronto para ser usado no script de fine-tuning.


Ajustes no fine-tuning

In [ ]:
!pip install --upgrade transformers datasets accelerate

In [ ]:
import os
import torch
import nltk
import numpy as np
import evaluate
from datasets import load_from_disk, DatasetDict
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments # Use TrainingArguments
)
import argparse

# --- Função para Calcular Métricas (ROUGE) ---
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    print("Baixando tokenizador 'punkt' do NLTK...")
    nltk.download("punkt", quiet=True)

rouge_metric = evaluate.load("rouge")
tokenizer_for_metrics = None


def compute_metrics(eval_preds):
    """Calcula métricas ROUGE para avaliação."""
    global tokenizer_for_metrics
    if tokenizer_for_metrics is None:
        print("AVISO: Tokenizer não definido para compute_metrics.")
        return {}

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    labels = np.where(labels != -100, labels, tokenizer_for_metrics.pad_token_id)

    decoded_preds = tokenizer_for_metrics.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer_for_metrics.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value * 100 if isinstance(value, float) else value for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer_for_metrics.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# --- Lógica Principal do Fine-tuning ---

def main(processed_dataset_path, model_name, output_model_dir, logging_dir, epochs, batch_size, grad_accum, lr, fp16_enabled):
    """Função principal para carregar dataset processado e treinar o modelo."""

    global tokenizer_for_metrics

    print("--- Iniciando Fine-tuning ---")

    # 1. Carregar Dataset Processado do Disco
    print(f"🔄 Carregando dataset pré-processado de '{processed_dataset_path}'...")
    if not os.path.exists(processed_dataset_path):
        print(f"❌ Erro: Diretório do dataset processado não encontrado em '{processed_dataset_path}'.")
        print("   Certifique-se de executar o script 'prepare_dataset.py' primeiro.")
        return

    try:
        tokenized_datasets = load_from_disk(processed_dataset_path)
        if not isinstance(tokenized_datasets, DatasetDict) or "train" not in tokenized_datasets:
             raise ValueError("O dataset carregado não é um DatasetDict válido ou não contém a chave 'train'.")
        train_dataset = tokenized_datasets["train"]
        eval_dataset = tokenized_datasets.get("test", None)
        print("✅ Dataset processado carregado.")
        if eval_dataset:
            print(f"   Encontrados {len(train_dataset)} exemplos de treino e {len(eval_dataset)} de teste.")
        else:
            print(f"   Encontrados {len(train_dataset)} exemplos de treino. (Sem dataset de teste)")

    except Exception as e:
        print(f"❌ Erro ao carregar dataset de '{processed_dataset_path}': {e}")
        return

    # 2. Carregar Modelo e Tokenizer Base
    print(f"🔄 Carregando modelo e tokenizer base: {model_name}...")
    try:
        tokenizer = T5Tokenizer.from_pretrained(model_name)
        model = T5ForConditionalGeneration.from_pretrained(model_name)
        tokenizer_for_metrics = tokenizer
        print("✅ Modelo e tokenizer carregados.")
    except Exception as e:
        print(f"❌ Falha ao carregar modelo/tokenizer {model_name}: {e}")
        return

    # 3. Data Collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        label_pad_token_id=tokenizer.pad_token_id,
        pad_to_multiple_of=8 if fp16_enabled else None
    )

    # 4. Argumentos de Treinamento (COM todos os args de avaliação e geração aqui para transformers 4.51.3)
    fp16_actual = fp16_enabled and torch.cuda.is_available()
    if fp16_enabled and not torch.cuda.is_available():
        print("⚠️ Aviso: FP16 solicitado, mas CUDA não está disponível. Treinando em FP32.")

    training_args = TrainingArguments(
        output_dir=output_model_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size, # per_device_eval_batch_size geralmente fica aqui
        gradient_accumulation_steps=grad_accum,
        learning_rate=lr,
        fp16=fp16_actual,
        weight_decay=0.01,
        save_total_limit=2,
        save_strategy="steps",
        save_steps=500,
        logging_steps=100,
        report_to="tensorboard",
        logging_dir=logging_dir,
        # --- Argumentos de avaliação e geração ESTÃO AQUI para transformers 4.51.3 ---
        evaluation_strategy="steps" if eval_dataset else "no",
        eval_steps=500 if eval_dataset else None,
        load_best_model_at_end=True if eval_dataset else False,
        metric_for_best_model="eval_loss" if eval_dataset else None,
        greater_is_better=False if eval_dataset else None,
        predict_with_generate=True,
        generation_max_length=150,
    )

    # 5. Inicializar o Trainer (APENAS com args que PERMANECEM aqui - compute_metrics)
    # Note: Você verá um FutureWarning sobre 'tokenizer', mas o erro TypeError é o que queremos corrigir.
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args, # Passa o objeto TrainingArguments
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer, # Continua aqui por enquanto (FutureWarning)
        data_collator=data_collator,
        compute_metrics=compute_metrics if eval_dataset else None, # compute_metrics permanece aqui
        # --- Argumentos de avaliação e geração NÃO ESTÃO AQUI ---
    )

    # 6. Iniciar Treinamento
    print("🚀 Iniciando treinamento...")
    try:
        train_result = trainer.train()
        print("✅ Treinamento concluído.")

        trainer.log_metrics("train", train_result.metrics)
        trainer.save_metrics("train", train_result.metrics)
        trainer.save_state()

        print(f"💾 Salvando modelo final em '{output_model_dir}'...")
        trainer.save_model(output_model_dir)
        print("✅ Modelo e tokenizer salvos.")

        if eval_dataset:
            print("📊 Realizando avaliação final no dataset de teste...")
            eval_metrics = trainer.evaluate()
            trainer.log_metrics("eval", eval_metrics)
            trainer.save_metrics("eval", eval_metrics)
            print(f"   Métricas de Avaliação Final: {eval_metrics}")
            print("✅ Avaliação final concluída.")

    except Exception as e:
        print(f"❌ Erro durante o treinamento/avaliação: {e}")
        # Adicionar um salvamento de checkpoint aqui pode ser útil em caso de erro

if __name__ == "__main__":
    # Mantendo o uso de argparse.Namespace para valores fixos
    args = argparse.Namespace(
        processed_dataset_path="./processed_dataset_jardinagem",
        model_name="google/flan-t5-large",
        output_model_dir="./modelo_jardinagem_ajustado",
        logging_dir="./logs_jardinagem",
        epochs=4,
        batch_size=4,
        grad_accum=2,
        lr=5e-5,
        fp16=False # ou True
    )

    # Chama a função main() com os argumentos
    main(
        args.processed_dataset_path,
        args.model_name,
        args.output_model_dir,
        args.logging_dir,
        args.epochs,
        args.batch_size,
        args.grad_accum,
        args.lr,
        args.fp16
    )

    print("\n🎉 Processo de Fine-tuning Finalizado!")

Baixando tokenizador 'punkt' do NLTK...


--- Iniciando Fine-tuning ---
🔄 Carregando dataset pré-processado de './processed_dataset_jardinagem'...
✅ Dataset processado carregado.
   Encontrados 18 exemplos de treino e 4 de teste.
🔄 Carregando modelo e tokenizer base: google/flan-t5-large...


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Modelo e tokenizer carregados.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
import transformers
print(transformers.__version__)

4.51.3


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Carregar o modelo e o tokenizer ajustados
modelo_path = "./modelo-ajustado"  # Caminho para o modelo ajustado
tokenizer = T5Tokenizer.from_pretrained(modelo_path)
model = T5ForConditionalGeneration.from_pretrained(modelo_path)

# Função para gerar a resposta com base na pergunta
def gerar_resposta(pergunta):
    # Ajustar o formato do prompt para melhorar a qualidade da resposta
    input_text = f"Como posso plantar tomates? {pergunta}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Gerar a resposta com configurações ajustadas
    with torch.no_grad():
        summary_ids = model.generate(inputs["input_ids"], max_length=250, num_beams=5, length_penalty=1.5, early_stopping=True)

    # Decodificar e retornar a resposta
    resposta = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return resposta

# Pergunta sobre como plantar tomate
pergunta = "Como plantar tomate?"

# Gerar a resposta
resposta = gerar_resposta(pergunta)

# Exibir a resposta
print(f"Pergunta: {pergunta}")
print(f"Resposta: {resposta}")


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './modelo-ajustado'.